# Property Appreciation Estimation and Recommendation for Strategic Real Estate Investments


## **Introduction**


### Problem Statement and Motivation

The U.S. housing market is influenced by a multitude of factors—historical housing prices, macroeconomic indicators (e.g., mortgage rates, GDP), seasonal variations, and even political cycles. For real estate investors, identifying locations with the highest potential returns and understanding the trajectory of property values is critical. This project aims to forecast property value trends at the zip code level and determine the top areas to invest in over a one-year horizon. 

### Objectives

    1.Examine Influential Factors:
Investigate the effect of mortgage rates, seasonal patterns, and political/election cycles correlate with property value trends.

    2.Forecast Housing Price Changes:
Use time series analysis and regression models to predict short-term (1 month), medium-term (1 quarter), and longer-term (1 year) changes in property values.

    3.Identify Optimal Investment Regions:
Pinpoint top U.S. zip codes that are likely to yield the highest return on investment over a one-year period.


## Data


### Data sources

1. Zillow Housing Data (Zillow Home Value Index - ZHVI and Zillow Home Value Forecast - ZHVF):
	
	- Description: 
	 	* The ZHVI serves as a critical metric, representing the typical home value and tracking market changes for homes within the 35th to 65th percentile range. This data is available in both a smoothed, seasonally adjusted format and a raw measure, allowing for different types of analysis. Our dataset includes house prices on a monthly basis for every zip code in the United States, providing comprehensive coverage at various levels of geographical granularity—county, city, state, and zip code. 
	 	* The ZHVF provides us with A month-ahead, quarter-ahead and year-ahead forecast of the Zillow Home Value Index (ZHVI). ZHVF is created using the all homes, mid-tier cut of ZHVI and is available both raw and smoothed, seasonally adjusted. We use these forecasts as the target variables to measure the performacne of our models.
	- Size: Approximately 26,338 rows × 305 columns.
	
	- Coverage: County, city, state, and zip code-level granularity spanning over two decades.
	
	- Reference: Zillow’s official website (obtained via downloadable CSV). https://www.zillow.com/research/data/:


In [ ]:
df2 = pd.read_csv("Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv")
print(df2.shape)
df2 = df2.sort_values(by="RegionName")
df2

2. Mortgage30US Dataset (FRED - Federal Reserve Economic Data):
	- Description: 30 year monthly fixed mortgage rates in the US from 2000 to 2024.
	- Size: Approximately 1,297 rows × 2 columns.
	- Reference:Federal Reserve Bank Website (obtained via downloadable CSV) https://fred.stlouisfed.org/series/MORTGAGE30US

In [ ]:
mort_df = pd.read_csv("MORTGAGE30US.csv")
# Plotting a line plot
fig = px.line(mort_df, x='DATE', y='MORTGAGE30US', title='24-Year Fixed Mortgage Rate Over Time')
# Show the plot
fig.show()

### Data Cleaning



In [ ]:
# Convert date columns to datetime if needed and sort by date
date_columns = df2.columns[9:] # the first 9 columns are metadata
df2[date_columns] = df2[date_columns].apply(pd.to_datetime, errors='coerce')

# Handle missing values - for example, forward fill NaN values
df2.fillna(method='ffill', inplace=True)
df2.fillna(method='bfill', inplace=True)

# Convert datetime columns to numeric values (e.g., Unix timestamp) for pct_change calculation
df2[date_columns] = df2[date_columns].apply(lambda x: x.view(np.int64))

# Calculate monthly percent change for property values for each ZIP code
df2_pct_change = df2.groupby('State')[date_columns].pct_change(axis=1)

# Adding zipcode column to the df1_pct_change
df2_pct_change.insert(0, 'State', df2['State'])

# Display the trend over time for each ZIP code
print(df2_pct_change)

# Save the results to a CSV file
df2_pct_change.to_csv("df2_pct_change.csv", index=False)

## Exploratory Data Analysis and Visualization

- **Adjusting Mortgage Rate Data (Flipping Across the Z-Axis)**: Given that mortgage rates are inversely related to property appreciation (i.e., as mortgage rates increase, property prices generally decrease), we flipped the mortgage rate data across the Z-axis. This transformation allowed for a direct, more meaningful comparison of how fluctuations in mortgage rates correlated with changes in property values, making the analysis more intuitive for understanding their relationship.

## Exploratory Data Analysis and Visualization

### Influence of Macroeconomic Indicators
**Observation**: The median percentage change in property values (from ZHVI – Zillow Home Value Index) across different states considering the dataset's granularity on zip codes stays extremely positive (+) during the end of 2020 through 2021. This was observed from the interactive choropleth map.

![EDA1_1](/home/armaan10/Desktop/UIC_MS/coursework/CS_418/project-check-in-team/Images/EDA1_1.png)
    
**Assumption**: Mortgage rates reached record lows during the latter half of 2020 and into early 2021. This is the same period during the Covid pandemic, when the demand for housing kept increasing. There are macroeconomic indicators that can explain this trend.

**Worked out**:
- Averaged out the percentage change in property values across states to get a single trend representing the whole of the U.S.
- Observed various macroeconomic indicators like mortgage rates, GDP, Federal interest rates during the period and tried to fit a factor that can tightly reason for the trend.
                                                                                                                                                 
                                                                                                                                                 
**Finding**:
- The inverse graph of mortgage rates is fitting the average percentage increase trend in property values across the U.S.


![EDA1_8](/home/armaan10/Desktop/UIC_MS/coursework/CS_418/project-check-in-team/Images/EDA1_8.png)
    


### Seasonal Trend Analysis
**Introduction**: In this analysis we investigate potential seasonal trends in average house prices over the years. Using raw data with a datetime column, we categorized each entry into seasons (Fall, Winter, Spring, Summer) based on the month. This approach allows us to analyze any fluctuations in house prices across seasons over a long period.
    
**Methodology**: 
 1.	Data Preparation: The raw data contained a datetime column, which was converted into rows. We then added a new column, "Season," categorizing each month into Fall, Winter, Spring, or Summer.
 2. Graph Analysis: After adding the seasonal column, we plotted a line graph to compare the average house prices for each season over the years. The graph below illustrates these seasonal trends.

**Observations**:
- The chart displays the average house prices over the years from around 2000 to 2025.
- Each line represents house prices for a different season (Fall, Spring, Summer, Winter).
- There’s a general upward trend in average house prices over time, with notable dips and peaks.
                                                                                                                                                 
![EDA2_1](/home/armaan10/Desktop/UIC_MS/coursework/CS_418/project-check-in-team/Images/EDA2_1.png)
                                                                                                                                                 
**Assumptions**:
- The dataset used for this chart includes sufficient historical data on seasonal average house prices, possibly sourced from a reliable real estate database.
- The chart aims to show whether there’s a seasonal effect on house prices, looking for distinct price differences between seasons over the years.

**Findings**:
- Similar Trends Across Seasons: All four seasons display similar patterns in price increases and decreases over time, with minimal differences between them, especially after 2015.
- Peaks and Troughs: There’s a peak in prices around 2005, a dip after 2007, a steady low around 2010, and then significant growth from around 2015 onward.
- Minor Seasonal Differences: Seasonal variation appears minor, with all seasonal lines staying close to each other across the years.

**Conclusion**:
- The average house prices appear to follow a consistent pattern across seasons, with no significant seasonal effect observed in recent years. Prices largely vary based on broader economic cycles rather than seasonal factors, suggesting that, at least in this dataset, seasonality doesn’t have a strong impact on average house prices over the years.

### Elections vs House Prices

- In the years 2000-2008, during the Republican Administration under Geoge W Bush, it was considered as the period of economic expansion. Interest rates were low, the availability of credit was easy, and minimal regulations around mortgages led to a housing boom which encouraged the investments in real estate. The American Dream Downpayment Act made it easier for people to own homes.
- 
But during the global financial crisis in 2008, this steep decline in home prices across the US as foreclosures surged and the housing dema d dropped, that created the economic downturn.
- In 2008-2016, during the Democratic Administration under Obama, the ARRA (American Recovery and Reinvestment Act) in 2009 aimed to stimulate the economic growth and support recovery, ideally it tried to stabilize the situation which can be seen from the graph. The recovery started to restore confidence in the economy. Priority was also given to affordable housing and assistance, though they had a limited effect on curbing the overall rise in house prices due to high ongoing demand and slow supply growth
- During the Republican administration in 2016 - 2020 under Trump underwent through a major policy, Tax Cuts and Jobs Act, that had provisions for lowering taxes for some individuals, which boosted disposable income for many and eventually made the housing market bit more welcoming, which made the house prices rise and that can be seen from the upward trend.
- During 2020-2021, mortgage rates were at their all time low, also covid brought demand in housing which caused a rise in demand, subsequently rise in property rates.

**Prediction based on recent elections:**
- In the 2024 fall, the feds have decreased the interest rates for the first time after 2020, also the elections held in November 2024, the republican party has come into power, substituting the democrats. Predicting the previous trends, it can be projected to be increasing ahead, since fed rates are low, this leads to lower mortgage rates increasing the housing demand and higher house rates.


![EDA2_1](/home/armaan10/Desktop/UIC_MS/coursework/CS_418/project-check-in-team/Images/EDA3_1.png)






## ML Analysis
